In [1]:
import os
import shutil
import sys
from collections import Counter
from copy import deepcopy
from pathlib import Path

from rich import print

abs_module_path = Path("./../../").resolve()
if (abs_module_path.exists()) and (str(abs_module_path) not in sys.path):
    sys.path.append(str(abs_module_path)) # add path to scan customized module

from modules.data.dataset import dsname
from modules.shared.utils import create_new_dir
from modules.shared.config import load_config
from modules.shared.pathnavigator import PathNavigator

In [ ]:
config = load_config("dataset.toml")
print(config)

dataset_seed_dir: str = config["dataset"]["seed_dir"]
dataset_data: str = config["dataset"]["data"]
dataset_palmskin_result: str = config["dataset"]["palmskin_result"]
dataset_base_size: str = config["dataset"]["base_size"]

In [3]:
path_navigator = PathNavigator()
dataset_cropped = path_navigator.dbpp.get_one_of_dbpp_roots("dataset_cropped_v3")

src_root = dataset_cropped.joinpath(dataset_seed_dir,
                                    dataset_data,
                                    dataset_palmskin_result,
                                    dataset_base_size)

In [ ]:
glob_pattern: str = "**/*.seg2.png" # Modify to find the desired file

img_paths = []
for dataset in ["test", "valid", "train"]:
    img_paths.extend(list(src_root.joinpath(dataset).glob(glob_pattern)))
img_paths = sorted(img_paths, key=dsname.get_dsname_sortinfo)

len(img_paths), img_paths

In [5]:
dst_dir = Path(r"") # Add directory for storing the collected images
create_new_dir(dst_dir)

for img_path in img_paths:
    shutil.copy(img_path, dst_dir)